Convert a PIT product to a datamodel
===
This notebook will walk through converting an example PIT product to a roman DataModel.

In [1]:
from astropy.io import fits
from astropy.time import Time
import astropy.units

import asdf
import gwcs
import numpy as np

Example file
===
For this example we'll use a slightly modified file generated by [pyimcom](https://github.com/Roman-HLIS-Cosmology-PIT/pyimcom) from the HLIS PIT.
The file here is modified to have small all-zero data arrays to make it small enough to include in the repository. The metadata is unmodified.

In [2]:
ff = fits.open("shear_zero.fits")
primary_header = ff[0].header
primary_header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                    1                                                  
NAXIS2  =                    1                                                  
NAXIS3  =                    3                                                  
EXTEND  =                    T                                                  
WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =                 80.5 / Pixel coordinate of reference point            
CRPIX2  =              10099.5 / Pixel coordinate of reference point            
PC1_1   = -1.0525173611111E-05 / Coordinate transformation matrix element       
PC2_2   =  1.1176215277778E-05 / Coordinate transformation matrix element       
CDELT1  =                  1

Converting file contents
===
Roman DataModels are stored in [ASDF](https://www.asdf-format.org/en/latest/) files. For our example file we first need to map the FITS structures (extensions and headers) to the ASDF tree structure.

For our example data file there are 2 extensions with image data and headers. FITS is a flat/linear structure which differs from the hierarchical/tree ASDF structure. In mapping the FITS contents to ASDF we'll want to utilize:
- the hierachical structure to capture the relationship between metadata values
- the ability of ASDF to represent high level objects with tags

To start, let's read the FITS WCS from the primary header and convert it to a [GWCS](https://gwcs.readthedocs.io/en/latest/).

In [3]:
wcs = gwcs.WCS(gwcs.utils.make_fitswcs_transform(primary_header), "pixel", "world")

Below we will put this `wcs` in the ASDF tree. Next let's map some of the additional FITS data to ASDF.

ASDF data can be nested and is not limited to short names. For our example file there are a number of FITS keywords that appear to relate to information about the source file. Let's collect this in a dictionary and nest the data in a way that captures the relationship between the keywords. We'll also use longer names (`filename_stem` vs `STEM`) to provide more information about the keywords.

In [4]:
source_file_meta = {
    "filename_stem": primary_header["STEM"],
    "block": {
        "x": primary_header["BLOCKX"],
        "y": primary_header["BLOCKY"],
    },
}

Similarly, there are a couple keywords related to the interpolation (`UMAX` and `SMAX`). Looking at the pyimcom code these correspond to the "leakage" and "noise" so we'll use those more descriptive names for ASDF.

In [5]:
interpolation_meta = {
    "leakage": primary_header["UMAX"],
    "noise": primary_header["SMAX"],
}

In FITS arrays are often stored in several keycards. Software then has to reconstruct the array manually after reading each of these keycards and lose some precision (due to the float->string->float conversion).
ASDF does not have either of these limitations and in this case the full-precision jacobian and covariance arrays can be stored in the ASDF tree.

In [6]:
jacobian = np.array([
    [primary_header["JXX"], primary_header["JXY"]],
    [primary_header["JYX"], primary_header["JYY"]],
])
covariance = np.array([primary_header["COVXX"], primary_header["COVXY"], primary_header["COVYY"]])

In the example file both `SIGMAOUT` and `PIXSCALE` keycards have comments that suggest that these values have specific units. Since the FITS standard says that software should not use these units we'll hard-code the units while mapping this data to ASDF.
In ASDF we can use quantities to associate values and units. The produced quantities will be validated (against an associated schema) and saved to the file. Software that processes this ASDF file can then key off the quantity tag and know the unit that is associated with the value.

In [7]:
sigma_out = astropy.units.Quantity(primary_header["SIGMAOUT"], astropy.units.arcsec)
pixel_scale = astropy.units.Quantity(primary_header["PIXSCALE"], astropy.units.arcsec)

Finally, there is a collection of keywords that contain information about the shear.

In [8]:
# Finally some keycards contain information about the shear
shear_parameters = {
    "oversampling": primary_header["OVERSAMP"],
    "applied_amplification": primary_header["MU"],
    "component_1": primary_header["ETA1"],
    "component_2": primary_header["ETA2"],
    "rotation_angle": primary_header["JROTATE"],
    "reduced_component_1": primary_header["G1"],
    "reduced_component_2": primary_header["G2"],
    "convergence_kappa": primary_header["CONV"],
    "jacobian": jacobian,
    "covariance": covariance,
    "sigma_out": sigma_out,
    "pixel_scale": pixel_scale,
}

Assembling the ASDF tree
===
Now that we've deconstructed the FITS file and mapped it's contents we can construct the ASDF tree. The mapping and tree construction was divided into two steps here to simply the example. In practice there is no need to separate these two steps.

In [9]:
# Make the ASDF tree
af = asdf.AsdfFile({
    "meta": {
        "wcs": wcs,
        "source_file": source_file_meta,
        "interpolation": interpolation_meta,
        "shear": shear_parameters,
    },
    "data": ff[0].data,
    "mask": ff[1].data,  # name inferred from looking at code
})

Recall from above that `source_file_meta` is a nested dictionary. Since we put this under the `meta` dictionary to access the `filename_stem` we'll need to traverse the ASDF tree.

In [10]:
af["meta"]["source_file"]["filename_stem"]

'https://irsa.ipac.caltech.edu/data/theory/openuniverse2024/roman/preview/RomanWAS/images/coadds/H158/Row{1:02d}/prod_H_{0:02d}_{1:02d}^'

Note how this nested hierarchy provides context to understand what was previously `STEM` in the FITS file. For any ASDF file it can be helpful to use [info](https://www.asdf-format.org/projects/asdf/en/latest/api/asdf.AsdfFile.html#asdf.AsdfFile.info) to see the often nested tree of data.

In [11]:
af.info(max_rows=None, max_cols=None)

root (AsdfObject)
├─meta (dict)
│ ├─wcs (WCS)
│ ├─source_file (dict)
│ │ ├─filename_stem (str): https://irsa.ipac.caltech.edu/data/theory/openuniverse2024/roman/preview/RomanWAS/images/coadds/H158/Row{1:02d}/prod_H_{0:02d}_{1:02d}^
│ │ └─block (dict)
│ │   ├─x (int): 18
│ │   └─y (int): 14
│ ├─interpolation (dict)
│ │ ├─leakage (float): 9.26145826696256e-11
│ │ └─noise (float): 0.11086250569005081
│ └─shear (dict)
│   ├─oversampling (float): 1.0
│   ├─applied_amplification (float): 1.0009008107296566
│   ├─component_1 (float): 0.06001800972625295
│   ├─component_2 (float): -7.350086350245e-18
│   ├─rotation_angle (float): 0.0
│   ├─reduced_component_1 (float): 0.030000000000000002
│   ├─reduced_component_2 (float): -3.673940397442e-18
│   ├─convergence_kappa (float): 0.0
│   ├─jacobian (ndarray)
│   │ ├─shape (tuple)
│   │ │ ├─[0] (int): 2
│   │ │ └─[1] (int): 2
│   │ └─dtype (Float64DType): float64
│   ├─covariance (ndarray)
│   │ ├─shape (tuple)
│   │ │ └─[0] (int): 3
│   │ └─dtype (

Converting to a DataModel
===
Now that we have the file contents mapped to ASDF we can work on making a DataModel for the data. This will involve:
- defining a schema to check/constrain the data
- registering this schema with roman_datamodels

Since the above changes are implemented by modifying [rad](https://github.com/spacetelescope/rad) and [roman_datamodels](https://github.com/spacetelescope/roman_datamodels) this example will only work with the following branches:
- https://github.com/braingram/rad/tree/shear_image
- https://github.com/braingram/roman_datamodels/tree/shear_image

First let's look at the added schema by asking asdf to load the resource by providing it's [URI](https://en.wikipedia.org/wiki/Uniform_Resource_Identifier).

In [12]:
resource = asdf.get_config().resource_manager["asdf://stsci.edu/datamodels/roman/schemas/PIT/HLIS/shear_image-1.0.0"]
print(resource.decode("ascii"))

%YAML 1.1
---
$schema: asdf://stsci.edu/datamodels/roman/schemas/rad_schema-1.0.0
id: asdf://stsci.edu/datamodels/roman/schemas/PIT/HLIS/shear_image-1.0.0

title: Shear image HLIS PIT product

datamodel_name: ShearImageModel

allOf:
  - $ref: asdf://stsci.edu/datamodels/roman/schemas/PIT/ccsp_mast_metadata-1.0.0
  - type: object
    properties:
      meta:
        properties:
          source_file:
            properties:
              filename_stem:
                type: string
                title: Input filename stem used to construct input mosaic
              block:
                title: Input block coordinates
                properties:
                  x:
                    type: integer
                  y:
                    type: integer
                required: [x, y]
            required: [filename_stem, block]
          interpolation:
            properties:
              leakage:
                type: number
              noise:
                type: number
       

See the asdf docs for:
- information about [how asdf uses URIs](https://www.asdf-format.org/projects/asdf/en/latest/asdf/extending/uris.html)
- [asdf schemas](https://www.asdf-format.org/projects/asdf/en/latest/asdf/extending/schemas.html)

In the above schema note that:
- the common `asdf://stsci.edu/datamodels/roman/schemas/PIT/ccsp_mast_metadata-1.0.0` schema is referenced
- the ASDF structure is described (and constrained) by the schema (for example `filename_stem` must be a string)

The addition of this schema to [RAD](https://github.com/spacetelescope/rad) and a small modification to [roman_datamodels](https://github.com/spacetelescope/roman_datamodels) allows us to use this schema for a new DataModel `ShearImageModel`. Let's create a new instance of that model with the tree we constructed above.

In [13]:
import roman_datamodels.datamodels as rdm
model = rdm.ShearImageModel.create_from_model(af.tree)
model.info(max_rows=None, max_cols=None)

root (AsdfObject)
└─roman (ShearImage)
  ├─meta (dict)
  │ ├─wcs (WCS) # WCS object
  │ ├─source_file (dict)
  │ │ ├─filename_stem (str): https://irsa.ipac.caltech.edu/data/theory/openuniverse2024/roman/preview/RomanWAS/images/coadds/H158/Row{1:02d}/prod_H_{0:02d}_{1:02d}^ # Input filename stem used to construct input mosaic
  │ │ └─block (dict) # Input block coordinates
  │ │   ├─x (int): 18
  │ │   └─y (int): 14
  │ ├─interpolation (dict)
  │ │ ├─leakage (float): 9.26145826696256e-11
  │ │ └─noise (float): 0.11086250569005081
  │ ├─shear (dict)
  │ │ ├─oversampling (float): 1.0
  │ │ ├─applied_amplification (float): 1.0009008107296566
  │ │ ├─component_1 (float): 0.06001800972625295
  │ │ ├─component_2 (float): -7.350086350245e-18
  │ │ ├─rotation_angle (float): 0.0 # Rotation angle
  │ │ ├─reduced_component_1 (float): 0.030000000000000002
  │ │ ├─reduced_component_2 (float): -3.673940397442e-18
  │ │ ├─convergence_kappa (float): 0.0
  │ │ ├─jacobian (ndarray)
  │ │ │ ├─shape (tuple)

Although we've constructed a `ShearImageModel` if we validate it we will find that validation fails.

In [14]:
try:
   model.validate()
except asdf.exceptions.ValidationError as err:
    print(f"ValidationError({err.message})")

ValidationError('mast_metadata' is a required property)


If we tried to save this model we would encounter the same `ValidationError` as asdf validates prior to saving.
This validation failed because we have not filled in the required `mast_metadata` (required by the `ccsp_mast_metadata-1.0.0` schema). For this example we'll fill in mostly junk values to produce a valid model.

In [15]:
model["mast_metadata"] = {
  'ccsp_id': 'HLIS',
  'ccsp_pi': 'na',
  'doi': 'na',
  'file_version': 'v0.0.0',
  'file_date': Time.now(),
  'license': 'na',
  'license_url': 'na',
  'start_time': Time("2025-09-01T00:00:00", format="isot"),
  'end_time': Time("2025-09-01T00:00:00", format="isot"), 
  'start_time_mjd': Time(60919.0, format="mjd"),
  'end_time_mjd': Time(60919.0, format="mjd"),
  'effective_exposure_time': -1,
}
model.validate()

Now that our model is valid we can save it to an ASDF file.

In [16]:
model.save("shear_model.asdf")

PosixPath('shear_model.asdf')